<h1> <b> Lab-02: Robot Programming </b> </h1>
<h4><b> Group Syntax:</b> </h4>

```
Yash Tarpara      - 202201422
Kaushik Prajapati - 202201472
```



In [ ]:
!pip install roboticstoolbox-python

<h4><b> Example-01:</b> </h4>

In [ ]:
import roboticstoolbox as rtb

# Define the robot model
robot = rtb.models.Panda()

# Print the robot model
print(robot)

# Visulise the robot
robot.plot(robot.qr, block=True, backend='pyplot')

<h4><b> Example-02:</b> </h4>

In [ ]:
# Display Robot with Swift
import roboticstoolbox as rtb
import swift

# Create swift instance
env =swift.Swift()
env.launch(realtime=True)

# Define the robot model
robot = rtb.models.Panda()
robot.q = robot.qr

robot.qd = [0.1, 0, 0, 0, 0, 0, 0.1]

# Add robot to swift
env.add(robot)

for _ in range(100):
    env.step(0.05)

# Stop the browser tab from closing
env.hold()

<h4><b> Example-03:</b> </h4>

In [ ]:
# Set goal position

import roboticstoolbox as rtb
import swift
import numpy as np
import spatialmath as sm
import spatialgeometry as sg

# Create swift instance
env =swift.Swift()
env.launch(realtime=True)

# Define the robot model
robot = rtb.models.Panda()
robot.q = robot.qr

# Add robot to swift
env.add(robot)

# Set goal position
goal = robot.fkine(robot.q) * sm.SE3.Tx(0.2) * sm.SE3.Ty(0.2) * sm.SE3.Tz(0.35)
axes = sg.Axes(length=0.1, base=goal)
env.add(axes)

# Arrived at a destination flag
arrived = False

# Time step
dt = 0.01

while not arrived:
    # v is a 6 vector representing the spatial error
    v, arrived = rtb.p_servo(robot.fkine(robot.q), goal, gain=1, threshold=0.01)
    J = robot.jacobe(robot.q)
    robot.qd = np.linalg.pinv(J) @ v

    # Step the environment
    env.step(dt)

# Stop the browser tab from closing
env.hold()

<h4><b> Example-04:</b> </h4>

In [ ]:
# Collotion geometry

import roboticstoolbox as rtb
import swift
import numpy as np
import spatialmath as sm
import spatialgeometry as sg

# Create swift instance
env =swift.Swift()
env.launch(realtime=True)

# Define the robot model
robot = rtb.models.Panda()
robot.q = robot.qr

# Add robot to swift
env.add(robot, robot_alpha=0.5, collision_alpha=0.5)

# Set goal position
goal = robot.fkine(robot.q) * sm.SE3.Tx(0.2) * sm.SE3.Ty(0.2) * sm.SE3.Tz(0.35)
axes = sg.Axes(length=0.1, base=goal)
env.add(axes)

# Arrived at a destination flag
arrived = False

# Time step
dt = 0.01

while not arrived:
    # v is a 6 vector representing the spatial error
    v, arrived = rtb.p_servo(robot.fkine(robot.q), goal, gain=0.1, threshold=0.01)
    J = robot.jacobe(robot.q)
    robot.qd = np.linalg.pinv(J) @ v

    # Step the environment
    env.step(dt)

# Stop the browser tab from closing
env.hold()

<h4><b> Example-05:</b> </h4>

In [ ]:
import swift
import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np

env = swift.Swift()
env.launch(realtime=True)

panda = rtb.models.Panda()
panda.q = panda.qr

Tep = panda.fkine(panda.q) * sm.SE3.Trans(0.2, 0.2, 0.45)

arrived = False
env.add(panda)

dt = 0.005

while not arrived:

    v, arrived = rtb.p_servo(panda.fkine(panda.q), Tep, 1)
    panda.qd = np.linalg.pinv(panda.jacobe(panda.q)) @ v
    env.step(dt)

# Uncomment to stop the browser tab from closing
env.hold()

<h4><b> Example-06:</b> </h4>

In [ ]:
import roboticstoolbox as rtb
from spatialmath import SE3
import matplotlib
import PIL.Image

# Step-1: Load a model of the Franka-Emika Panda robot defined by a URDF file
robot = rtb.models.Panda()
#print(robot)


# Step-2: Forward Kinematics
Te = robot.fkine(robot.qr)  # forward kinematics
#print(Te)

# Step-3: Inverse Kinematics
Tep = SE3.Trans(0.6, -0.3, 0.1) * SE3.OA([0, 1, 0], [0, 0, -1])
sol = robot.ik_LM(Tep)         # solve IK
#print(sol)

# Step-4: FK shows that desired end-effector pose was achieved
q_pickup = sol[0]
#print(robot.fkine(q_pickup))

# Step-5: Trajectory plot
qt = rtb.jtraj(robot.qr, q_pickup, 50)
#robot.plot(qt.q, backend='pyplot', movie='panda1.gif')

# Step-6: Plot the trajectory in the Swift simulator
robot.plot(qt.q)